**Write the names of all group members (max. 5 members)**:
- FirstName1, LastName1
- FirstName2, LastName2
- FirstName3, LastName3
- FirstName4, LastName4
- FirstName5, LastName5

When submitting your work, please follow closely the template below.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch.utils.data import DataLoader, TensorDataset

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

import math 

# Exercise 1

### Question 1.a

Check that $ S_t = s_0 \exp \left( \sigma W_t - \frac{1}{2} \sigma^2 t \right)$ solves the Black-Scholes SDE.


The general Black-Scholes SDE is given by

$$dS_t = rS_t dt + \sigma S_t dW_t,\quad S_0=s_0\in\mathbb{R}_+ $$

Itô's lemma states that any general (well defined) SDE on the form

$$dX_t = \mu(t,X_t) dt +  \sigma(t,X_t) dW_t$$

can be solved with 

$$ df(t,X_t) = \left(\frac{\partial f}{\partial t}+\mu(t,X_t) \frac{\partial f}{\partial x}+\frac{\sigma^2(t,X_t)}{2} \frac{\partial^2 f}{\partial x^2}\right)dt +\sigma(t,X_t)\frac{\partial f}{\partial x} dW_t$$

let $f(t,X_t) = \ln(X_t)$

$$d(\ln(X_t)) = \left(\mu(t,X_t) \frac{1}{X_t}-\frac{\sigma^2(t,X_t)}{2} \frac{1}{X_t^2}\right)dt +\sigma(t,X_t)\frac{1}{X_t} dW_t$$

and integrate over $[0,t]$ w.r.t. s:

$$\int^{t}_{0} d(\ln(X_s)) = \displaystyle \int^{t}_{0} \left(\mu(s,X_s) \frac{1}{X_s}-\frac{\sigma^2(s,X_s)}{2} \frac{1}{X_t^2}\right)ds + \displaystyle \int^{t}_{0}\sigma(s,X_s)\frac{1}{X_s} dW_s$$

$$ \Leftrightarrow  \ln\left(\frac{X_t}{X_0}\right) = \left(\mu(t,X_t) \frac{1}{X_t}-\frac{\sigma^2(t,X_t)}{2} \frac{1}{X_t^2}\right)t + \sigma(t,X_t)\frac{1}{X_t} W_t $$

Let $\mu(t,X_t) \coloneqq r S_t$, $\sigma(t,X_t) \coloneqq \sigma S_t$ and $X_t \coloneqq S_t$ (as in the formula at top)

$$ \Rightarrow  \ln\left(\frac{S_t}{S_0}\right) = \left(rS_t-\frac{\sigma^2}{2}\right)t + \sigma W_t  \Leftrightarrow S_t = S_0 \exp\left(\left(rS_t-\frac{\sigma^2}{2}\right)t + \sigma W_t\right)$$

And if we assume $r = 0$ then 

$$ S_t = S_0 \exp\left(\sigma W_t-\frac{\sigma^2 t}{2} \right) \quad \textbf{q.e.d}$$





### Question 1.b - Generate dataset

In [ ]:
torch.manual_seed(10)

N = 30
s0 = 1
m = 10_000 # training set size
n = 1_000 # test set size
sigma = 0.5
T = 30/365

Z = torch.randn(m+n,N)
a = - (sigma**2)/2 * (T/N) * torch.ones(N)

S = torch.zeros(m+n,N)
S[:,0] = torch.ones_like(S[:,0])
for t in range(N-1):
    b = sigma*(T/N)**(0.5) * Z[:,t+1]
    S[:,t+1] = S[:,t]*torch.exp(a[t]+b)

# Generate the training set
S_train = S[:m,:]
# Generate the test set
S_test = S[m:,:]

plt.hist(S_train.mean(dim=1),density=True)
plt.hist(S_test.mean(dim=1),density=True)
plt.show()
plt.hist(S_train.std(dim=1),density=True)
plt.hist(S_test.std(dim=1),density=True)
plt.show()

### Question 1.c - Model implementation

In [ ]:
# Implement the deep hedging model
nh1 = 12
nh2 = 12

H = [] 
for t in range(N):
    network = nn.Sequential(
    nn.Linear(1, nh1),
    nn.ReLU(), 
    nn.Linear(nh1, nh2), 
    nn.ReLU(), 
    nn.Linear(nh2, 1))

    H.append(network)

# You may use and edit the code shown in demo.ipynb

### Question 1.d - Model training

In [ ]:
K = 1
S0 = 1
# Implement the payoff function of a call option
g = lambda s,k: torch.relu(s-k)
Phi = lambda x: 0.5*(1+math.erf(x/math.sqrt(2)))
# Implement a function that returns the Black-Scholes price of a call option with given strike and maturity
def C(S_t,t=0,r=0,K=1,T=1,sigma=0.5):
    d_p = 1/(sigma*math.sqrt(T-t)) * (math.log(S_t/K)+(r+sigma**2/2)*(T-t))
    d_m = d_p - sigma*(T-t)**(0.5)
    return Phi(d_p)*S_t-Phi(d_m)*K*math.exp(-r*(T-t))

y_true = g(S[:,-1],K)-C(S0)
y_train = y_true[:m]
y_test = y_true[m:]

n_epochs = 10
batch_size = 16
# Train the model
params = []
for t in range(N):
    params += list(H[t].parameters())
opt = torch.optim.Adam(params)
L = []

for epoch in range(n_epochs):
    epoch_loss = 0
    indices = torch.randperm(m)
    for b in range(0, m, batch_size):
        batch_idx = indices[b:b+batch_size]
        S_batch = S[batch_idx]    
        y_batch = y_true[batch_idx]

        opt.zero_grad()
        
        hedge_sum = 0
        for t in range(N-1):
            h_t = H[t](S_batch[:, t:t+1])
            dS_t = S_batch[:, t+1] - S_batch[:, t]
            hedge_sum += h_t.squeeze() * dS_t

        loss = ((y_batch - hedge_sum)**2).mean()
        loss.backward()
        opt.step()
        opt.zero_grad()
        
        epoch_loss += loss.item()
    
    L.append(epoch_loss)

plt.plot(L)
plt.show()

### Question 1.e - Model evaluation

In [ ]:
# Evaluate the hedging losses on the test set

# Plot a histogram of the hedging losses and print mean and standard deviation

### Question 1.f(i) - Compute analytical solution

Show a **step-by-step** derivation of the analytical hedging strategy:

\begin{align*}
H^{\text{BS}}_t(s) & = \frac{\partial C(s, t)}{\partial s} \\
& = \ldots
\end{align*}

No points for solutions without explicit derivation.

### Question 1.f(ii) - Losses comparison with analytical solution

In [ ]:
# Implement the formula you found

# Evaluate the losses of the analytical hedging strategies on the test set

# Plot a histograms of the hedging losses (analytical and deep hedging) and print means and standard deviations

How does your deep hedging model compare with the analytical solution? If it's worse, you must improve its performance. A few questions for you to consider...

Have you trained your deep hedging model well?
- Are the losses centered on zero? ($10^{-5} \approx 0$)
- Is their standard deviation comparable with that of the analytical solution?

How can you improve performance?
- train for more epochs
- try a different learning rate
- change the networks' architectures (first try more hidden neurons, then try more hidden layers)
- try different batch sizes
- use a learning rate scheduler (see https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate)
- try a different activation function (e.g. $\tanh$)
- implement regularization (use the `weight_decay` argument in the optimizer)
- implement batch-normalization (see https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html)
- experiment with different optimizers
- ask in the forum

### Question 1.g - Learned strategy comparison with analytical solution

In [ ]:
# Plot both the deep hedging strategies and the analytical hedging strategies

- For what times $t_j$ are the two functions most similar? 
- Why?

# Exercise 2

### Question 2.a - Generate dataset

In [ ]:
# Write function to sample V
# Write function to sample S

# Generate the training set
# Generate the test set

### Question 2.b - Model implementation

In [ ]:
# Implement the deep hedging model for a generic alpha

### Question 2.c - Model training

In [ ]:
# Train two deep hedging models (for alpha=0.5 and alpha=0.99) for the call option

### Question 2.d - Price comparison

In [ ]:
# Print the prices p for alpha=0.5 and alpha=0.99
# Which one is higher? Why? 

### Question 2.e - Price comparison

In [ ]:
# Plot a histograms of the hedging losses (for alpha=0.5 and alpha=0.99) and print means and standard deviations